## Setup

In [ ]:
from helpers_functions.load import *

archive = "/archive"

### Check structure

In [ ]:
folder_structure(archive)


### Split train_val

In [ ]:
train_val_split(archive)


## Data Preprocessing

### Create class-label linking

In [ ]:
from helpers_functions.one_hot import *

class_label_str_to_idx, class_label_idx_to_str = create_class_label_dict()

### One-hot encoding

In [ ]:
img_idx = "00000001_000"
vec = one_hot_encode(xml_file)
print(vec)


### 1.2.2 Training a Classifier

#### Make a Dataloader

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import csv
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader

class MedicalImageDataset(Dataset):
    def __init__(self, base_dir, transform=None):

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        return image, label, img_name


# Example:
dataset = MedicalImageDataset()

loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Check one batch
for imgs, labels, names in loader:
    print("Image batch shape:", imgs.shape)
    print("Label batch shape:", labels.shape)
    print("First image name:", names[0])
    print("First label vector:", labels[0])
    
    # Display the first image in the batch
    plt.imshow(imgs[0].permute(1,2,0))
    plt.title(f"Image: {names[0]}")
    plt.axis('off')
    plt.show()
    break


#### Normalize our Dataset

In [ ]:
from helpers_functions.normalize import *

# Example:
img = ""
mean, std = normalize(img)


#### Load pretrained_model

#### Train the CNN

#### Classifier Performance